# Simulation

In [1]:
indexes = [
    ("A","1"), ("B","1"), ("C", "1"),
    ("A","2"),            ("C", "2"),
    ("A","3"), ("B","3"), ("C", "3"),
    ("A","4"), ("B","4"),
];

### Simulate number of reads
The code below generates read multiplicities for each index combination, in accordance with what would be expected for a set of mis-assignment probabilities. The simulation logic assumes that the index1 and index2 mis-assignment events are independent.  Wright and Vetsigian found a higher probability than what would be expected for independent variables, but the double mis-assignment rate is still very low -- negligible for practical purposes:

|                  |              |
|------------------|--------------|
|Incorrect i5:     |   0.0604 %   |
|Incorrect i7:     |   0.0955 %   |
|Incorrect sequence|   0.0872 %   |
|Multiple incorrect|   0.0003 %   |

It is also notable that the i7 and i5 (respectively, the first and second index) mis-assignment rates are significantly different.

We cannot detect sequence misassignments at all using demultiplexing data, and most double misassignments will not be detectable.

In [28]:
sample_read_mean = 10e6               # Number of reads
sample_read_sd = sample_read_mean*0.5 # Statistical standard deviation (normal dist.)
index1_misassign_rate = 0.0955 / 100.0
index2_misassign_rate = 0.0604 / 100.0
sequence_misassign_rate = 0.0872 / 100.0

0.0008719999999999999

In [30]:
true_sample_reads = Dict([index => (randn()*sample_read_sd + sample_read_mean) for index = indexes]);

In [35]:
index1s = Set([i1 for (i1,i2) = indexes])
index1_multiplicity = Dict([i1,sum(1 for i1,2) for (i1,i2)=indexes])
index2s = Set([i2 for (i1,i2) = indexes])

Set(Any["4","1","2","3"])

In [24]:


sim_known_sample_reads = Dict()
sim_unknown_sample_reads = Dict()

for index in indexes
    # Generate all match-ups
    total_reads = total_sample_reads[index]
    i1, i2 = index
    
end

### Setup for analysis

In [7]:
i1s = Set()
sample_single_indexes = map(Set, zip(indexes...))

2-element Array{Set{String},1}:
 Set(String["B","A","C"])    
 Set(String["4","1","2","3"])